# What Happens When Transitions are Non-Linear?
In the previous notes, one of the fundamental assumptions of the Kalman filter is that both state transition and sensor states are linear functions of the current state. This... is problematic: almost **ALL** systems are inherently non-linear! Let's look at an example non-linear transformation:

<img src="images/NonLinearExample.jpg" alt="Non-Linear Transform Example from Textbook" width="400px" />

Let's assume that $x$ is a Gaussian R.V., and it is transformed via some non-linear function resulting in $y=g(x)$. The resultant distribution of $y$ will **not** be Gaussian! In these cases, we can still *approximate* the resultant distribution into the form of a Gaussian through many different methods. Simply, we could run a Monte-Carlo experiment of draws of $x$ and get a large dataset of $y$ values. Estimating the first two moments from the dataset (that is, computing $\mu_y, \sigma_y^2$), we could simply use those as the parameters for a Gaussian $\hat{y} \sim N(\mu_y, \sigma_y)$.

Another method is to apply a linearization technique to the transform...

# Introducing the Extended Kalman Filter
The key here is to linearize the transform via *[Taylor Expansion](https://en.wikipedia.org/wiki/Taylor_series#Definition)* to a first-order approximation; that is to say:

$$
g(x) \approx g(a) + g'(a)(x-a)
$$

One issue faced with this approximation is the virtue that the slope of this linearization is completely dependent upon the value $a$ we select. A logical conclusion is to set $a = \mu_{t-1}$, which adjusts our transformation to

$$
g(u_t, x_{t-1}) \approx g(u_t, u_{t-1}) + G_t (x_{t-1} - \mu_{t-1}) \\
G_t = g'(u_t, u_{t-1}) := \frac{\partial g(u_t, x_{t-1})}{\partial x_{t-1}}
$$

Note that $G_t$ is the *Jacobian* of our transformation when $x \in \mathbb{R}^n$. Given this matrix, we can linearlly transform the covariance (note, this is still an approximation), and thus only slightly modify our original Kalman algorithm.


## The Extended Kalman Algorithm
**ExtendedKalmanFilter**($\mu_{t-1}, \Sigma_{t-1}, u_t, z_t$):
* Predict $\mu_t, \Sigma_t$
  * $\overline{\mu_t} = g(u_t, \mu_{t-1})$
  * $\overline{\Sigma_t} = G_t \Sigma_{t-1} G_t^\top + R_t$
* Correct estimate of $\mu_t, \Sigma_t$
  * $K_t = \overline{\Sigma_t} H_t^\top (H_t \overline{\Sigma_t} H_t^\top + Q_t)^{-1}$
  * $\mu_t = \overline{\mu_t} + K_t \cdot (z_t - h(\mu_t))$
  * $\Sigma_t = (I - K_t H_t) \overline{\Sigma_t}$
  
Note that $H_t$ may also be the Jacobian of the transform $z_t = h(x_t)$. That is to say either transform can remain linear!

# The Unscented Kalman Filter
TODO